In [1]:
import sys
import os
src_dir = os.path.join(os.getcwd(), '../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

from nn_model import *
from run_nn import *
from train_nn import *
from dataloader import *

import pandas as pd

print(os.getcwd())


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


/Users/isomorphicdude/Library/Mobile Documents/com~apple~CloudDocs/MSc/SB2.2_Statistical_Machine_Learning/Practicals/Assessed/SML_pratical/notebooks


In [2]:
# Load the training data and the test inputs
X_all = pd.read_csv('../data/X_train.csv', index_col = 0, header=[0, 1, 2]) # inputs of the training set
y_all = pd.read_csv('../data/y_train.csv', index_col = 0).squeeze('columns') # outputs of the training set
X_test = pd.read_csv('../data/X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set

In [3]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA_model = LinearDiscriminantAnalysis(n_components=None)

In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
y_all = LabelEncoder().fit_transform(y_all)

In [14]:
from sklearn.model_selection import train_test_split
# create dataset
X_train, X_val, y_train, y_val = train_test_split(X_all, 
                                                  y_all, 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  shuffle=False)

# convert to numpy
X_train = X_train.to_numpy()
X_train = LDA_model.fit_transform(X_train, y_train)
X_val = X_val.to_numpy()
X_val = LDA_model.fit_transform(X_val, y_val)

In [15]:
from torch.utils.data import DataLoader

train_set = MusicData(X_train, y_train)
val_set = MusicData(X_val, y_val)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)

X shape: torch.Size([4800, 7])
y shape: torch.Size([4800])
X shape: torch.Size([1200, 7])
y shape: torch.Size([1200])


In [16]:
import numpy as np
mlp = MLP(input_dim=X_train.shape[1], 
          hidden_dims=[128, 64, 16], 
          output_dim=np.unique(y_all).shape[0],
          dropout=0.0)

train_loss, test_loss = run_training_loop(
    num_epochs=1000,
    train_loader=train_loader,
    val_loader=val_loader,
    model=mlp,
    optimizer=torch.optim.Adam(mlp.parameters(), lr=0.1),
    loss_function=torch.nn.CrossEntropyLoss(),
    device="cpu",
)

INFO:root:Path not provided, saving model to models
INFO:root:Epoch: 1, train Loss: 1.3538, val Loss: 1.5959, val Accuracy: 43.61%
INFO:root:Saving model...
INFO:root:Epoch: 6, train Loss: 1.2508, val Loss: 1.8491, val Accuracy: 40.52%
INFO:root:Epoch: 11, train Loss: 1.2170, val Loss: 1.8183, val Accuracy: 40.30%
INFO:root:Epoch: 16, train Loss: 1.1902, val Loss: 1.7073, val Accuracy: 39.83%
INFO:root:Epoch: 21, train Loss: 1.1584, val Loss: 1.7289, val Accuracy: 42.19%
INFO:root:Epoch: 26, train Loss: 1.1992, val Loss: 1.9038, val Accuracy: 37.61%
INFO:root:Epoch: 31, train Loss: 1.1693, val Loss: 1.8185, val Accuracy: 43.01%
INFO:root:Epoch: 36, train Loss: 1.1629, val Loss: 1.5950, val Accuracy: 46.27%
INFO:root:Saving model...
INFO:root:Epoch: 41, train Loss: 1.1454, val Loss: 1.7706, val Accuracy: 48.08%
INFO:root:Epoch: 46, train Loss: 1.1518, val Loss: 1.8004, val Accuracy: 45.67%
INFO:root:Epoch: 51, train Loss: 1.1538, val Loss: 1.5213, val Accuracy: 47.40%
INFO:root:Saving m

KeyboardInterrupt: 

In [8]:
from sklearn.metrics import accuracy_score
# Load model state dict
mlp.load_state_dict(torch.load('models/model.pth'))
mlp.eval()

# predict
y_pred = mlp(torch.tensor(X_val).float()).detach().numpy()

accuracy_score(y_val, np.argmax(y_pred, axis=1))

0.10833333333333334